In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.round(np.random.uniform(1,100,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.sqrt(torch.var(gt30_dummy_data_tensor, correction = 1))
dummy_data_mean = torch.mean(gt30_dummy_data_tensor)

In [6]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        num_fil_X = torch.sum((X>30).double())
        fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        fil_std_X = torch.tensor([ele[0] if ele[0]>30 else self.data_mean for ele in X[0]])
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/278749945.py:11: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/278749945.py:11: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/278749945.py:11: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if y

In [7]:
# Prover/ data owner side
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
theory_output = torch.sqrt(torch.var(gt30_data_tensor, correction = 1))
data_mean = torch.mean(gt30_data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
        
    def forward(self,X):
        num_fil_X = torch.sum((X>30).double())
        fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
        x_mean_cons = torch.abs(torch.sum(fil_mean_X)-num_fil_X*(self.data_mean))<=torch.abs(0.01*num_fil_X*self.data_mean)
        fil_std_X = torch.tensor([ele[0] if ele[0]>30 else self.data_mean for ele in X[0]])
        return (torch.logical_and(torch.abs(torch.sum((fil_std_X-self.data_mean)*(fil_std_X-self.data_mean))-self.w*self.w*(num_fil_X-1))<=torch.abs(0.02*self.w*self.w*(num_fil_X-1)),x_mean_cons),self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [2], "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/1125113063.py:15: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/1125113063.py:15: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_mean_X = torch.tensor([ele[0] if ele[0]>30 else 0 for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/1125113063.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning i

theory output:  tensor(12.7586)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-161232,28184],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1531,"total_const_size":10,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/1125113063.py:17: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  fil_std_X = torch.tensor([ele[0] if ele[0]>30 else self.data_mean for ele in X[0]])
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_90366/1125113063.py:17: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  fil_std_X = torch.tensor([ele[0] if ele[0]>30 else self.data_mean for ele in X[0]])


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2
spawning module 0


==== setting up ezkl ====


spawning module 2


Time setup: 25.42341899871826 seconds
Theory output:  tensor(12.7586)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.75
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12362648763242643187, 13940026059969050459, 6027715406760125980, 2781413989188023337]]], 'proof': '02ecac8771c25429cb97b0c77e17c8feaef23d31ae3f04d0b7f10928db0ddba02b2ad7e8663816d7a53156717244a5bc3405c6fd4a8172e738bdddb12fb7c2f2181dc584ec2853d593cfba9186743315f9dcd1dcc367c90c8545bd0a24182b311d28ce0e515b6a06c2169fff5defb339f537fde1c60b7d1f6a5c7275ff18185e09bb583af239a39477c969a52b7cf7db251e8564aaccf1dcfa366928d042642911991cb23f723681c2e7a23c01efb8daa10018f46fe8539f5fb5da23fc3e15032be23f0df8d833bd95372af1ba67796e15ae4412629b0d1cb2f619597a146a6819a60cb3c3d881b2e8acd707c08a20cd946626f9c39a107ca1f214eaac4db4731e13f89e8d1793608bdd5704f83e1cba9d52bef43574ad8e506b28b63b5872022279edb85ad4dbe9e1d3f91941fb0a9462b117c9cef36539fb2bdee36bd8f7662c44c7173e7d4d3a54fe9ca4a6959870e9e2a6d242841e418d162e755279

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[10537101196673941533, 17227541574925932677, 11187715152301828262, 1869164017182098189], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12362648763242643187, 13940026059969050459, 6027715406760125980, 2781413989188023337]]]
proof boolean:  1.0
proof result 1 : 12.75
verified
